# Download

This notebook downloads the required data files for preprocessing.

## Libraries

In [1]:
import requests
import zipfile

import pandas as pd

from datetime import datetime, timedelta
from IPython.display import display, HTML
from pathlib import Path
from time import time

## Functions

In [2]:
# Downloads a file from a link
def download(url, file):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(file, 'wb') as f:
            for chunk in r.iter_content(1024):
                f.write(chunk)

# Unzips a single compressed .zip file
def unzip(file, folder):
    ext = Path(file).suffixes[0].lower()
    with zipfile.ZipFile(file, 'r') as z:
        if ext == '.shp':
            for target in z.infolist():
                zext = ''.join(Path(target.filename).suffixes).replace(ext, '')
                target.filename = Path(file).stem + zext
                z.extract(target, folder)
        else:
            target = z.infolist()[0]
            target.filename = Path(file).stem
            z.extract(target, folder)
        
# Formats human readable sizes in bytes
# https://stackoverflow.com/questions/1094841/get-human-readable-version-of-file-size
def format_size(num, suffix="B"):
    for unit in ["", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"]:
        if abs(num) < 1024.0:
            return f"{num:3.1f} {unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f} Yi{suffix}"

# Extracts file info for summary
def file_info(file):
    stats = Path(file).stat()
    modified = datetime.fromtimestamp(stats.st_mtime)
    created = datetime.fromtimestamp(stats.st_birthtime)
    size = format_size(stats.st_size)
    out = {'created': created, 'modified': modified, 'size': size}
    return out

## Settings

In [3]:
folder = '../tmp/downloads'
data_sources = pd.read_csv('data.csv')

## Run

Run the download process for all defined data files in the settings.

In [4]:
# Create folder if not exists
Path(folder).mkdir(parents=True, exist_ok=True)

### Download Data

Download data from url sources to ``folder``.

In [5]:
# Start download
print(f'Starting Downloads ({datetime.now()})...')
start = time()
for row in data_sources.itertuples():
    
    # Data file vars
    url = row.url
    file = f'{folder}/{row.file}'
    
    # Download data file if it does not exist otherwise skip
    if not Path(file).is_file():
        print(f'Downloading {row.file} ({datetime.now()})...')
        download(url, file)
        print(f'Downloaded {row.file} ({datetime.now()})')
    else:
        print(f'Skipping {row.file} - file exists ({datetime.now()})...')
        
# End downloads
end = time()
elapsed = str(timedelta(seconds=end - start))
print(f'Downloads Complete ({datetime.now()})')
print(f'Elapsed Time ({elapsed})')

Starting Downloads (2022-02-18 23:35:11.927028)...
Downloaded toronto.geojson (2022-02-18 23:35:13.809686)
Downloaded centrelines.geojson.zip (2022-02-18 23:35:14.678552)
Downloaded collisions.geojson (2022-02-18 23:35:16.041388)
Downloaded traffic.csv.zip (2022-02-18 23:35:16.761494)
Downloaded autospeed_enforcement.geojson (2022-02-18 23:35:17.473642)
Downloaded watch_your_speed.geojson (2022-02-18 23:35:18.778127)
Downloaded red_light_cams.geojson (2022-02-18 23:35:19.783277)
Downloaded police.geojson (2022-02-18 23:35:20.411435)
Downloaded ambulance.geojson (2022-02-18 23:35:21.236640)
Downloaded fire_hydrants.geojson (2022-02-18 23:35:40.414805)
Downloaded fire_stations.geojson (2022-02-18 23:35:41.444869)
Downloaded renewables.geojson (2022-02-18 23:35:42.374441)
Downloaded bicycle_parking.geojson (2022-02-18 23:35:58.009605)
Downloaded transit_shelters.geojson (2022-02-18 23:36:05.638579)
Downloaded wayfind.geojson (2022-02-18 23:36:06.747880)
Downloaded litter.geojson (2022-02-

### Unzip Data

Unzip compressed data files.

In [6]:
# Unzip data files
print(f'Unzipping Data ({datetime.now()})...')
start = time()
for row in data_sources.itertuples():
    
    # Data file vars
    url = row.url
    file = f'{folder}/{row.file}'
    
    # Download data file if it does not exist otherwise skip
    path = Path(file)
    if path.suffix.lower() == '.zip':
        if path.with_suffix('').is_file():
            print(f'Skipping {row.file} - already unzipped ({datetime.now()})...')
        else:
            print(f'Unzipping {row.file}')
            unzip(file, folder)
            print(f'Unzipped {row.file} ({datetime.now()})')
    else:
        print(f'Skipping {row.file} - not a zip file ({datetime.now()})...') 

# End downloads
end = time()
elapsed = str(timedelta(seconds=end - start))
print(f'Unzip Complete ({datetime.now()})')
print(f'Elapsed Time ({elapsed})')

Unzipping Data (2022-02-18 23:36:30.257254)...
Skipping toronto.geojson - not a zip file (2022-02-18 23:36:30.257707)...
Unzipping centrelines.geojson.zip
Unzipped centrelines.geojson.zip (2022-02-18 23:36:30.593683)
Skipping collisions.geojson - not a zip file (2022-02-18 23:36:30.593838)...
Unzipping traffic.csv.zip
Unzipped traffic.csv.zip (2022-02-18 23:36:30.800898)
Skipping autospeed_enforcement.geojson - not a zip file (2022-02-18 23:36:30.800973)...
Skipping watch_your_speed.geojson - not a zip file (2022-02-18 23:36:30.800992)...
Skipping red_light_cams.geojson - not a zip file (2022-02-18 23:36:30.801012)...
Skipping police.geojson - not a zip file (2022-02-18 23:36:30.801028)...
Skipping ambulance.geojson - not a zip file (2022-02-18 23:36:30.801042)...
Skipping fire_hydrants.geojson - not a zip file (2022-02-18 23:36:30.801056)...
Skipping fire_stations.geojson - not a zip file (2022-02-18 23:36:30.801071)...
Skipping renewables.geojson - not a zip file (2022-02-18 23:36:30

## Summary

In [7]:
# Copy data sources to use in summary
summary = data_sources.copy()

# Get file infos and add to summary
info = [file_info(f'{folder}/{row.file}') for row in summary.itertuples()]
info = pd.DataFrame(info)
summary = pd.concat([summary, info], axis=1)

# Rearrange summary columns and display
summary = summary[['file', 'size', 'created', 'modified', 'source', 'source_url', 'url']]
display(HTML(summary.to_html(render_links=True)))

,file,size,created,modified,source,source_url,url
0,toronto.geojson,1.9 MiB,2022-02-18 23:35:13.574342,2022-02-18 23:35:13.809513,City of Toronto Open Data Portal,https://open.toronto.ca/dataset/neighbourhoods/,https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/a083c865-6d60-4d1d-b6c6-b0c8a85f9c15?format=geojson&projection=4326
1,centrelines.geojson.zip,13.6 MiB,2022-02-18 23:35:14.189826,2022-02-18 23:35:14.677475,City of Toronto Open Data Portal,https://open.toronto.ca/dataset/toronto-centreline-tcl/,https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/ba71b78e-464b-46dd-8185-a0d249fe4857
2,collisions.geojson,248.9 MiB,2022-02-18 23:35:15.195910,2022-02-18 23:35:16.041278,Toronto Police Service Public Safety Data Portal,https://data.torontopolice.on.ca/datasets/TorontoPS::traffic-collisions-asr-t-tbl-001/about,https://opendata.arcgis.com/api/v3/datasets/2a36d61277a84c3eb9d0beebef4c023d_0/downloads/data?format=geojson&spatialRefId=4326
3,traffic.csv.zip,8.1 MiB,2022-02-18 23:35:16.355065,2022-02-18 23:35:16.761260,City of Toronto Open Data Portal,https://open.toronto.ca/dataset/traffic-volumes-at-intersections-for-all-modes/,https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/0ffadfc9-b017-44df-a1b1-905591e54caa
4,autospeed_enforcement.geojson,25.4 KiB,2022-02-18 23:35:17.469039,2022-02-18 23:35:17.473240,City of Toronto Open Data Portal,https://open.toronto.ca/dataset/automated-speed-enforcement-locations/,https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/e25e9460-a0e8-469c-b9fb-9a4837ac6c1c?format=geojson&projection=4326
5,watch_your_speed.geojson,320.6 KiB,2022-02-18 23:35:18.710201,2022-02-18 23:35:18.777524,City of Toronto Open Data Portal,https://open.toronto.ca/dataset/school-safety-zone-watch-your-speed-program-locations/,https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/4e2221b9-da3a-4ef8-b8eb-17e95b7abaa0?format=geojson&projection=4326
6,red_light_cams.geojson,169.6 KiB,2022-02-18 23:35:19.740649,2022-02-18 23:35:19.782822,City of Toronto Open Data Portal,https://open.toronto.ca/dataset/red-light-cameras/,https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/b57a31a1-5ee6-43e3-bfb9-206ebe93066d?format=geojson&projection=4326
7,police.geojson,9.0 KiB,2022-02-18 23:35:20.408113,2022-02-18 23:35:20.410544,City of Toronto Open Data Portal,https://open.toronto.ca/dataset/police-facility-locations/,https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/4afc3c66-5614-466a-b714-e8d6336fc6d3?format=geojson&projection=4326
8,ambulance.geojson,37.6 KiB,2022-02-18 23:35:21.213185,2022-02-18 23:35:21.236339,City of Toronto Open Data Portal,https://open.toronto.ca/dataset/ambulance-station-locations/,https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/e49245ba-395c-46bf-bcf8-22fc7024d649?format=geojson&projection=4326
9,fire_hydrants.geojson,10.7 MiB,2022-02-18 23:35:39.972058,2022-02-18 23:35:40.414129,City of Toronto Open Data Portal,https://open.toronto.ca/dataset/fire-hydrants/,https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/beaaa552-6338-4c81-95be-411e6cef6b89?format=geojson&projection=4326
